In [25]:
from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from config import OWM_API_key as key, port, host
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, InvalidDocument, DuplicateKeyError
# import json
import os
from os import path


API_key = key
host = host
port = port
filename = 'resources/zip_list.csv'
global zipcode
owm = OWM(API_key) # the OWM object
global obs
global reception_time
global zid# = Zip_Location.get_ID()
global zlon# = location.get_lon()
global zlat# = location.get_lat()
global client

def set_location(code):
    ''' Get the latitude and longitude corrosponding to the zip code.
        
        :param code: the zip code to find weather data about
        :type code: string
    '''
    global obs, zlat, zlon
    obs = owm.weather_at_zip_code(f'{code}', 'us')
    location = obs.get_location()
    zlon = location.get_lon()
    zlat = location.get_lat()
    return


def current():
    ''' Dump the current weather to a json

        :return current: the currently observed weather data
        :type current: dict
    '''
    global obs
    current = obs.to_JSON() # dump all that weather data to a json object
    return(current)


def five_day():
    ''' Get each weather forecast for the corrosponding zip code. 
    
        :return five_day: the five day, every three hours, forecast for the zip code\
        :type five_day: dict
    '''
    global obs, zlat, zlon
    forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    forecast = forecaster.get_forecast()
    five_day = forecast.to_JSON()
    return(five_day)


# YOU DON'T HAVE THE API ACCESS TO GET THIS TYPE OF FORECAST REQUEST
# def daily():
#     ''' Fetch the daily forecast. '''
#     forecaster = owm.daily_forecast_at_coords(zlat, zlon)
#     forecast = forecaster.get_forecast()
#     daily = forecast.to_JSON()
#     return(daily)


def check_db_access(host, port):
    ''' A check that there is write access to the database
    
        :param host: the database host
        :type host: string
        :param port: the database connection port
        :type port: int
    '''
    client = MongoClient(host=host, port=port)
    try:
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print('Connection made')
    except ConnectionFailure:
        print("Server not available")
        return

    # check the database connections
        # Get a count of the number of databases at the connection (accessible through that port)
        # before attempting to add to it
    db_count_pre = len(client.list_database_names())
        # Add a database and collection
    db = client.test_db
    col = db.test_col

    # Insert something to the db
    post = {'name':'Chuck VanHoff',
           'age':'38',
           'hobby':'gardening'
           }
    col.insert_one(post)

        # Get a count of the databases after adding one
    db_count_post = len(client.list_database_names())

    if db_count_pre-db_count_post>=0:
        print('Your conneciton is flipped up')
    else:
        print('You have write access')

    client.drop_database(db)
    return(client)

def to_json(data, code):
    ''' Store the collected data as a json file in the case that the database
        is not connected or otherwise unavailable.
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param code: the zip code associated with the data from the list codes
        :type code: sting
    '''
    collection = data
    directory = os.getcwd()
    save_path = path.join(directory, 'Data', f'{code}.json')
    Data = open(save_path, 'a+')
    Data.write(collection)
    Data.close()
    return

def load(data, client):
    ''' Load the data to the database if possible, otherwise write to json file. 
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param client: the pymongo client object
        :type client: MongoClient
    '''
    if type(data) == dict:
        try:
            db = client.forcast
            col = db.code
            col.insert_one(data)
        except DuplicateKeyError:
            client.close()
            print('closed db connection')
            to_json(data, code)
            print('Wrote to json')
    else:
        print('data type is not dict')
        client.close()
        print('closed db connection')
    return

def write_list_to_file(zip_list, filename):
    """ Write the zip codes to csv file.
        
        :param zip_list: the list created from the zip codes dataframe
        :type zip_list: list stings
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "w") as z_list:
        for z in zip_list:
            z_list.write(f'{z},')
#         z_list.write(z for z in zip_list)
    return
        
def read_list_from_file(filename):
    """ Read the zip codes list from the csv file.
        
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "r") as z_list:
        return z_list.read().strip().split(',')

def get_weather(codes):
    ''' Get the weather from the API and load it to the database. 
    
    :param codes: list of zip codes
    :type codes: list of strings
    '''
    client = check_db_access(host, port)
    for code in codes:
        data = {}
        set_location(code)
        data.update({'zipcode': code,
                     'current': current(),
                     'five_day': five_day(),
                    })
        load(data, client)
    client.close()
    return

In [8]:
def make_zip_list():
    ''' Make a list of zip codes in the specified state.
        
        :param state: the two-letter abreviation of the state whose zip codes you'd like listed
        :type state: string
    '''
    import pandas as pd
    
    df = pd.read_csv("resources/zip_code_database.csv")
    # Make a datafram from the rows with the specified state and write the 'zip' column to a list 
#     zip_list = df.loc[lambda df: df['state'] == f'{state.upper()}']['zip'].tolist()
    zip_list = df['zip'].tolist()
    return(zip_list)

In [9]:
from pyowm.exceptions.api_response_error import NotFoundError

codes = make_zip_list() #read_list_from_file(filename)
n=0
successes=0
exceptions=0
zip_list = []

for code in codes:
        zipp = f'{code}'
        zip_list.append(zipp)
#     try:
#         print('try setting loation for ', zip_list[n])
#         set_location(code)
#         print(f'successfully set location for {code}')
#         successes+=1
# #         write_list_to_file(zip_list, 'resources/success_zips.csv')
#     except NotFoundError:
#         print("except", f'NotFoundError with zipcode {code}')
# #         write_list_to_file(zip_list, 'resources/exception_zips.csv')
#         exceptions+=1
#         pass
# #     n+=1
# print(f'successes = {successes}; exceptions = {exceptions}')
# get_weather(codes)

In [10]:
print(len(zip_list))

42632


In [11]:
# n=0
# for n in range(len(zip_list)):
#     try:
#         print('try', zip_list[n])
#         set_location(zip_list[n])
#         print(f'successfully set location for {zip_list[n]}')
#         successes+=1
# #         write_list_to_file(zip_list, 'resources/success_zips.csv')
#     except NotFoundError:
#         print("except", f'NotFoundError with zipcode {code}')
# #         write_list_to_file(zip_list, 'resources/exception_zips.csv')
#         exceptions+=1
#         pass

In [12]:
# for code in codes:
# #     data = {}
#     if n<5:
#         zipp = f'{code}'
# #         print(zipp)
#         zip_list.append(f'{code}')
#         print(f'appended {code} to zip_list')
#     else:
#         print(f'n is greater than 5')
#         break

In [18]:
# from socket import timeout
success_zips = []
fail_zips = []
for zipp in zip_list:
    if int(zipp) > 10000:
        try:
            print('try setting location for ', zipp)
            set_location(zipp)
            print(f'successfully set location for {zipp}', len(success_zips))
            success_zips.append(zipp)
            successes+=1
        except NotFoundError:
            print("except", f'NotFoundError with zipcode {zipp}')
            fail_zips.append(zipp)
            exceptions+=1
            pass
write_list_to_file(success_zips, 'resources/success_zips.csv')
write_list_to_file(fail_zips, 'resources/fail_zips.csv')
print(f'successes = {successes}; exceptions = {exceptions}, all written to files')

try setting location for  10001
successfully set location for 10001 0
try setting location for  10002
successfully set location for 10002 1
try setting location for  10003
successfully set location for 10003 2
try setting location for  10004
successfully set location for 10004 3
try setting location for  10005
successfully set location for 10005 4
try setting location for  10006
successfully set location for 10006 5
try setting location for  10007
successfully set location for 10007 6
try setting location for  10008
successfully set location for 10008 7
try setting location for  10009
successfully set location for 10009 8
try setting location for  10010
successfully set location for 10010 9
try setting location for  10011
successfully set location for 10011 10
try setting location for  10012
successfully set location for 10012 11
try setting location for  10013
successfully set location for 10013 12
try setting location for  10014
successfully set location for 10014 13
try setting loca

successfully set location for 10168 103
try setting location for  10169
successfully set location for 10169 104
try setting location for  10170
successfully set location for 10170 105
try setting location for  10171
successfully set location for 10171 106
try setting location for  10172
successfully set location for 10172 107
try setting location for  10173
successfully set location for 10173 108
try setting location for  10174
successfully set location for 10174 109
try setting location for  10175
successfully set location for 10175 110
try setting location for  10176
successfully set location for 10176 111
try setting location for  10177
successfully set location for 10177 112
try setting location for  10178
successfully set location for 10178 113
try setting location for  10179
successfully set location for 10179 114
try setting location for  10184
except NotFoundError with zipcode 10184
try setting location for  10185
successfully set location for 10185 115
try setting location for

successfully set location for 10532 210
try setting location for  10533
successfully set location for 10533 211
try setting location for  10535
successfully set location for 10535 212
try setting location for  10536
successfully set location for 10536 213
try setting location for  10537
successfully set location for 10537 214
try setting location for  10538
successfully set location for 10538 215
try setting location for  10540
successfully set location for 10540 216
try setting location for  10541
successfully set location for 10541 217
try setting location for  10542
successfully set location for 10542 218
try setting location for  10543
successfully set location for 10543 219
try setting location for  10545
successfully set location for 10545 220
try setting location for  10546
successfully set location for 10546 221
try setting location for  10547
successfully set location for 10547 222
try setting location for  10548
successfully set location for 10548 223
try setting location for

successfully set location for 10976 319
try setting location for  10977
successfully set location for 10977 320
try setting location for  10979
successfully set location for 10979 321
try setting location for  10980
successfully set location for 10980 322
try setting location for  10981
successfully set location for 10981 323
try setting location for  10982
successfully set location for 10982 324
try setting location for  10983
successfully set location for 10983 325
try setting location for  10984
successfully set location for 10984 326
try setting location for  10985
successfully set location for 10985 327
try setting location for  10986
successfully set location for 10986 328
try setting location for  10987
successfully set location for 10987 329
try setting location for  10988
successfully set location for 10988 330
try setting location for  10989
successfully set location for 10989 331
try setting location for  10990
successfully set location for 10990 332
try setting location for

successfully set location for 11359 424
try setting location for  11360
successfully set location for 11360 425
try setting location for  11361
successfully set location for 11361 426
try setting location for  11362
successfully set location for 11362 427
try setting location for  11363
successfully set location for 11363 428
try setting location for  11364
successfully set location for 11364 429
try setting location for  11365
successfully set location for 11365 430
try setting location for  11366
successfully set location for 11366 431
try setting location for  11367
successfully set location for 11367 432
try setting location for  11368
successfully set location for 11368 433
try setting location for  11369
successfully set location for 11369 434
try setting location for  11370
successfully set location for 11370 435
try setting location for  11371
successfully set location for 11371 436
try setting location for  11372
successfully set location for 11372 437
try setting location for

...

successfully set location for 29058 8470
try setting location for  29059
successfully set location for 29059 8471
try setting location for  29061
successfully set location for 29061 8472
try setting location for  29062
successfully set location for 29062 8473
try setting location for  29063
successfully set location for 29063 8474
try setting location for  29065
successfully set location for 29065 8475
try setting location for  29067
successfully set location for 29067 8476
try setting location for  29069
successfully set location for 29069 8477
try setting location for  29070
successfully set location for 29070 8478
try setting location for  29071
successfully set location for 29071 8479
try setting location for  29072
successfully set location for 29072 8480
try setting location for  29073
successfully set location for 29073 8481
try setting location for  29074
successfully set location for 29074 8482
try setting location for  29075
successfully set location for 29075 8483
try settin

successfully set location for 29316 8583
try setting location for  29318
except NotFoundError with zipcode 29318
try setting location for  29319
successfully set location for 29319 8584
try setting location for  29320
successfully set location for 29320 8585
try setting location for  29321
successfully set location for 29321 8586
try setting location for  29322
successfully set location for 29322 8587
try setting location for  29323
successfully set location for 29323 8588
try setting location for  29324
successfully set location for 29324 8589
try setting location for  29325
successfully set location for 29325 8590
try setting location for  29329
successfully set location for 29329 8591
try setting location for  29330
successfully set location for 29330 8592
try setting location for  29331
successfully set location for 29331 8593
try setting location for  29332
successfully set location for 29332 8594
try setting location for  29333
successfully set location for 29333 8595
try setting

successfully set location for 29482 8692
try setting location for  29483
successfully set location for 29483 8693
try setting location for  29484
successfully set location for 29484 8694
try setting location for  29485
successfully set location for 29485 8695
try setting location for  29486
successfully set location for 29486 8696
try setting location for  29487
successfully set location for 29487 8697
try setting location for  29488
successfully set location for 29488 8698
try setting location for  29492
successfully set location for 29492 8699
try setting location for  29493
successfully set location for 29493 8700
try setting location for  29501
successfully set location for 29501 8701
try setting location for  29502
successfully set location for 29502 8702
try setting location for  29503
successfully set location for 29503 8703
try setting location for  29504
successfully set location for 29504 8704
try setting location for  29505
successfully set location for 29505 8705
try settin

successfully set location for 29643 8805
try setting location for  29644
successfully set location for 29644 8806
try setting location for  29645
successfully set location for 29645 8807
try setting location for  29646
successfully set location for 29646 8808
try setting location for  29647
successfully set location for 29647 8809
try setting location for  29648
successfully set location for 29648 8810
try setting location for  29649
successfully set location for 29649 8811
try setting location for  29650
successfully set location for 29650 8812
try setting location for  29651
successfully set location for 29651 8813
try setting location for  29652
successfully set location for 29652 8814
try setting location for  29653
successfully set location for 29653 8815
try setting location for  29654
successfully set location for 29654 8816
try setting location for  29655
successfully set location for 29655 8817
try setting location for  29656
successfully set location for 29656 8818
try settin

APICallError: Exception in calling OWM Weather API.
Reason: {"cod":429, "message": "Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription http://openweathermap.org/price"}
Caused by: None

In [26]:
write_list_to_file(success_zips, 'resources/success_zips.csv')
write_list_to_file(fail_zips, 'resources/fail_zips.csv')

-73.98 40.78
